# TODO: Replica Exchange: Gaussian replicas over different means
This example uses `ReplicaExchange` for a scalar variable with Gaussian targets of different `μ`, and exchanges neighboring replicas.

In [1]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

using Random
using MonteCarloX

rng = MersenneTwister(2026)
nrep = 6
mus = collect(range(-3.0, 3.0, length=nrep))
replicas = [Metropolis(MersenneTwister(1000 + i), x -> -0.5 * (x - mus[i])^2) for i in 1:nrep]
re = ReplicaExchange(replicas; rng=rng)

x = copy(mus)
println("Initial states: ", x)

  Activating project at `~/.julia/dev/MonteCarloX/notebooks`


Initial states: [-3.0, -1.8, -0.6, 0.6, 1.8, 3.0]


In [2]:
function local_metropolis_step!(x::Vector{Float64}, replicas; proposal_sigma=0.7)
    for i in eachindex(x)
        alg = replicas[i]
        xnew = x[i] + proposal_sigma * randn(alg.rng)
        logr = alg.logweight(xnew) - alg.logweight(x[i])
        if accept!(alg, logr)
            x[i] = xnew
        end
    end
    return nothing
end

local_metropolis_step! (generic function with 1 method)

In [3]:
nsteps = 3000
for t in 1:nsteps
    local_metropolis_step!(x, replicas)
    phase = iseven(t) ? 0 : 1
    sweep_exchanges!(re, x; phase=phase)
end

println("Final states: ", x)
println("Exchange attempts: ", re.exchange_attempts)
println("Exchange accepted: ", re.exchange_accepted)
println("Exchange rate: ", exchange_rate(re))

Final states: [-2.4484713861316134, -1.7948234424201757, -2.2984601245707803, -0.035430890595358644, 2.3961246091670225, 1.9595262463799947]
Exchange attempts: 7500
Exchange accepted: 2923
Exchange rate: 0.3897333333333333


Expected behavior: replicas perform local random walks around their Gaussian means while neighboring exchanges let trajectories diffuse through `μ`-space.